# From raw measurements to compressed sensing measurements
    Now we have generated raw FLIM measurements from generate_raw_measurement, we can use the raw measurements to simulate the compressively sensed measurements.

## Some preparation

In [25]:
import scipy
import numpy as np
import matplotlib.pyplot as plt

A function to read in raw_measurement files into the memory

In [26]:
# This function reads the raw measurements data from the Raw_Measurements file
# Parameters:
# - starting_index is where the index starts
# - batch is the number of batches
# For example, read_raw_data(20,10) will return the raw measurement data from sample_20 to sample_30
# There are up to 8000 raw measurements data
def read_raw_data(starting_index, batch):
    raw_measurements = np.zeros([batch,128,128,256]).astype(np.uint8)
    for j in range(batch):
        i = j+starting_index
        raw_measurements[j,:,:] = scipy.io.loadmat('Data\Raw_Measurements\sample_'+str(i)+'.mat')['measurements']
    return raw_measurements

## Generate images
Compressive measurements are made by $y = \phi x$, where y is the compressively measured datasets, and x is the original image. $\phi$ is the measurement matrix
x in here is a 1D array, so we reshape the lifetime raw measurements into 

$x = [batch_i, t, y]$

So first of all we need a Hadamard Matrix, as our measurement matrix $\phi$

In [73]:
# raw_measurements = [batch_i, x, y, t]
hadamard= scipy.linalg.hadamard(128).reshape(-1)
print('The shape of reshaped hadamard matrix is ', hadamard.shape)

The shape of reshaped hadamard matrix is  (16384,)


In [82]:
raw_measurements = read_raw_data(0,1)
print('The shape of the raw measurements data is ',raw_measurements.shape)

The shape of the raw measurements data is  (1, 128, 128, 256)


In [83]:
# raw_measurements = [batch_i,x,y,t]
x_2D = np.moveaxis(raw_measurements,[1,2,3],[2,3,1])
# x_2D = [batch_1,t,x,y]
x = np.reshape(x_2D, (x_2D.shape[0],x_2D.shape[1],-1))
print('The shape of 1D measurements x is ', x.shape)

The shape of 1D measurements x is  (1, 256, 16384)
